In [ ]:
# --- 1. Imports & read files ---
import pandas as pd
import numpy as np

# modeling tools
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

RANDOM_STATE = 42

# Read Kaggle Titanic data (put CSVs in the same folder as this notebook)
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")
sample_sub = pd.read_csv("gender_submission.csv")

# Basic shapes
print(train.shape, test.shape)
train.head()


(891, 12) (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
# --- 2. Quick EDA: structure & missing values ---
# Basic info
train.info()

# Target distribution
print("\nTarget distribution:")
print(train['Survived'].value_counts(normalize=True))

# Missing values in both train and test
def missing_table(df, name):
    # Return missing count and ratio
    miss = df.isnull().sum()
    miss = miss[miss > 0].sort_values(ascending=False)
    if len(miss) == 0:
        print(f"No missing in {name}.")
        return
    rate = (miss / len(df)).round(3)
    out = pd.DataFrame({'missing_count': miss, 'missing_rate': rate})
    print(f"\nMissing values in {name}:")
    display(out)

missing_table(train, "train")
missing_table(test, "test")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB

Target distribution:
Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64

Missing values in train:


,missing_count,missing_rate
Cabin,687,0.771
Age,177,0.199
Embarked,2,0.002



Missing values in test:


,missing_count,missing_rate
Cabin,327,0.782
Age,86,0.206
Fare,1,0.002


In [ ]:
# --- 3. Combine for unified preprocessing/feature engineering ---
train['dataset'] = 'train'
test['dataset'] = 'test'
full = pd.concat([train, test], ignore_index=True)

# Helper: extract Title from Name (e.g., "Mr.", "Miss.", "Master.", "Mrs.")
def extract_title(name: str) -> str:
    # Extract title between comma and dot (e.g., "Braund, Mr. Owen" -> "Mr")
    title = name.split(',')[1].split('.')[0].strip()
    # Group rare titles
    rare_map = {
        'Mlle':'Miss', 'Ms':'Miss', 'Mme':'Mrs',
        'Lady':'Rare', 'Countess':'Rare', 'Capt':'Rare', 'Col':'Rare', 'Don':'Rare',
        'Dr':'Rare', 'Major':'Rare', 'Rev':'Rare', 'Sir':'Rare', 'Jonkheer':'Rare',
        'Dona':'Rare'
    }
    return rare_map.get(title, title)

# --- Cleaning ---
full['Title'] = full['Name'].apply(extract_title)
full['FamilySize'] = full['SibSp'] + full['Parch'] + 1
full['IsAlone'] = (full['FamilySize'] == 1).astype(int)

# Embarked & Fare & Age fills (same as before)
embark_mode = full['Embarked'].mode().iloc[0]
full['Embarked'] = full['Embarked'].fillna(embark_mode)
full['Fare'] = full.groupby('Pclass')['Fare'].transform(lambda s: s.fillna(s.median()))
age_map = full.groupby(['Title','Pclass'])['Age'].median()
full['Age'] = full.apply(lambda r: age_map.loc[r['Title'], r['Pclass']] if pd.isna(r['Age']) else r['Age'], axis=1)

# Interaction feature
full['Sex_Embarked'] = full['Sex'] + "_" + full['Embarked']

# Ticket prefix (first token letters-only; helps capture booking group effects)
def ticket_prefix(t):
    # Keep letters only from first token
    token = str(t).split()[0]
    letters = ''.join([c for c in token if c.isalpha()])
    return letters if letters else 'NONE'
full['TicketPrefix'] = full['Ticket'].apply(ticket_prefix)

# Fill Embarked with mode (very few missing in train)
embark_mode = full['Embarked'].mode().iloc[0]
full['Embarked'] = full['Embarked'].fillna(embark_mode)

# Fill Fare using median within Pclass (only 1 missing in test usually)
full['Fare'] = full.groupby('Pclass')['Fare'].transform(
    lambda s: s.fillna(s.median())
)

# Age imputation strategy:
# median Age within (Title, Pclass) groups works well
age_map = full.groupby(['Title','Pclass'])['Age'].median()
def fill_age(row):
    if pd.isna(row['Age']):
        return age_map.loc[row['Title'], row['Pclass']]
    return row['Age']
full['Age'] = full.apply(fill_age, axis=1)

# Sanity check after fills
missing_table(full, "full (after fills)")

# Define features to use
features = [
    'Pclass','Sex','Age','SibSp','Parch','Fare','Embarked',
    'Title','FamilySize','IsAlone','Sex_Embarked'
]
cat_cols = ['Sex','Embarked','Title','Sex_Embarked']
num_cols = ['Pclass','Age','SibSp','Parch','Fare','FamilySize','IsAlone']




Missing values in full (after fills):


,missing_count,missing_rate
Cabin,1014,0.775
Survived,418,0.319


In [ ]:
# --- 4. Build preprocessing + models with updated feature list ---
# 1) Split back
train_proc = full[full['dataset'] == 'train'].copy()
test_proc  = full[full['dataset'] == 'test'].copy()

# 2) Feature lists
features = [
    'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked',
    'Title', 'FamilySize', 'IsAlone', 'Sex_Embarked'
]
cat_cols = ['Sex', 'Embarked', 'Title', 'Sex_Embarked']
num_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone']

# 3) Build X/y with a safety check
X = train_proc[features].copy()
y = train_proc['Survived'].astype(int)
X_test = test_proc[features].copy()

# quick guard: make sure all columns exist
missing_in_X = [c for c in cat_cols + num_cols if c not in X.columns]
assert len(missing_in_X) == 0, f"Columns not found in X: {missing_in_X}"

# 4) Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), num_cols),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('ohe', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_cols),
    ],
    remainder='drop'
)

# 5) Models
RANDOM_STATE = 42
models = {
    'LogReg': LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    'RF'    : RandomForestClassifier(n_estimators=400, random_state=RANDOM_STATE),
    'GB'    : GradientBoostingClassifier(random_state=RANDOM_STATE),
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

cv_results = {}
for name, clf in models.items():
    pipe = Pipeline(steps=[('prep', preprocessor), ('clf', clf)])
    scores = cross_val_score(pipe, X, y, scoring='accuracy', cv=cv)
    cv_results[name] = (scores.mean(), scores.std())
    print(f"{name}: {scores.mean():.4f} ± {scores.std():.4f}")

best_name = max(cv_results, key=lambda k: cv_results[k][0])
print("Best:", best_name, cv_results[best_name])


LogReg: 0.8317 ± 0.0093
RF: 0.8126 ± 0.0134
GB: 0.8372 ± 0.0119
Best: GB (np.float64(0.8372481325717155), np.float64(0.011949678194556198))


In [16]:
# --- 5. Fit best model on full train and predict test ---
best_model = models[best_name]
final_pipe = Pipeline(steps=[('prep', preprocessor), ('clf', best_model)])
final_pipe.fit(X, y)

test_pred = final_pipe.predict(X_test).astype(int)

# Build submission file
submission = pd.DataFrame({
    'PassengerId': test_proc['PassengerId'].astype(int),
    'Survived': test_pred
})
submission.to_csv("submission.csv", index=False)
submission.head()


,PassengerId,Survived
891,892,0
892,893,0
893,894,0
894,895,0
895,896,1
